In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import multivariate_normal
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# Set random seed for reproducibility
rng = np.random.default_rng(42)

## 1. Data Laden en Voorbereiden

We laden eerst de California Housing dataset en selecteren een subset van features.

In [2]:
# Load California Housing dataset
housing = fetch_california_housing(as_frame=True)
df = housing.frame

# Select features and target
features = ["MedInc", "HouseAge"]
X = df[features].values
y = df["MedHouseVal"].values

print(f"Dataset shape: X={X.shape}, y={y.shape}")
print("\nFirst 5 samples:")
print(df[["MedInc", "HouseAge", "MedHouseVal"]].head())

Dataset shape: X=(20640, 2), y=(20640,)

First 5 samples:
   MedInc  HouseAge  MedHouseVal
0  8.3252      41.0        4.526
1  8.3014      21.0        3.585
2  7.2574      52.0        3.521
3  5.6431      52.0        3.413
4  3.8462      52.0        3.422


### Standardisatie

Voor Bayesiaanse regressie is het belangrijk om de data te **standardiseren** (mean=0, std=1). Dit maakt het makkelijker om informatieve priors te kiezen.

In [3]:
# Standardize features and target
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

print(f"Scaled X mean: {X_scaled.mean(axis=0)}")
print(f"Scaled X std: {X_scaled.std(axis=0)}")
print(f"Scaled y mean: {y_scaled.mean():.6f}")
print(f"Scaled y std: {y_scaled.std():.6f}")

Scaled X mean: [6.60969987e-17 5.50808322e-18]
Scaled X std: [1. 1.]
Scaled y mean: 0.000000
Scaled y std: 1.000000


### Subset voor Visualisatie

Voor de visualisatie nemen we een kleine subset van 50 observaties.

In [4]:
# Take a small subset for visualization
n_samples = 50
indices = rng.choice(len(X_scaled), size=n_samples, replace=False)
X_subset = X_scaled[indices]
y_subset = y_scaled[indices]

print(f"Working with {n_samples} samples for visualization")

Working with 50 samples for visualization


## 2. Bayesiaanse Parameterschatting Setup

We schatten drie parameters: $b_0$ (intercept), $b_1$ (effect van inkomen), $b_2$ (effect van leeftijd).

### Prior Distributie

We kiezen een **trivariate normaalverdeling** als prior:

$$
p(\pmb{b}) = \mathcal{N}(\pmb{\mu}_0, \pmb{\Sigma}_0)
$$

Voor gestandaardiseerde data kiezen we:
- Mean: $\pmb{\mu}_0 = [0, 0, 0]^T$
- Covariance: $\pmb{\Sigma}_0 = 0.5^2 \pmb{I}$ (redelijk brede prior)

In [5]:
# Prior parameters
prior_mean = np.array([0.0, 0.0, 0.0])  # [b0, b1, b2]
prior_cov = np.eye(3) * 0.5**2  # Independent priors with std=0.5

print("Prior mean:")
print(prior_mean)
print("\nPrior covariance:")
print(prior_cov)

Prior mean:
[0. 0. 0.]

Prior covariance:
[[0.25 0.   0.  ]
 [0.   0.25 0.  ]
 [0.   0.   0.25]]


### Likelihood Parameters

We nemen aan dat de **observatieruis** $\epsilon$ normaal verdeeld is met bekende standaarddeviatie $\sigma=0.8$ (in de gestandaardiseerde ruimte).

In [6]:
# Known noise standard deviation (in standardized space)
sigma = 0.8
beta = 1 / (sigma**2)  # precision parameter

print(f"Noise std (σ): {sigma}")
print(f"Precision (β): {beta}")

Noise std (σ): 0.8
Precision (β): 1.5624999999999998


## 3. Visualisatie Grid Setup

Voor visualisatie maken we een 2D grid in de $(b_1, b_2)$ ruimte, waarbij we $b_0$ op zijn prior mean (0) fixeren.

In [7]:
# Create a grid for visualization (focusing on b1 and b2)
b1_range = np.linspace(-2, 2, 100)
b2_range = np.linspace(-2, 2, 100)
B1, B2 = np.meshgrid(b1_range, b2_range)

# We'll fix b0 at its prior mean (0) for 2D visualization
b0_fixed = 0.0

## 4. Prior Visualisatie

We visualiseren eerst de **prior** distributie voor $(b_1, b_2)$.

In [8]:
# Compute prior density on the grid (marginalizing over b0)
# For visualization, we fix b0=0 and look at the 2D slice
prior_2d_mean = prior_mean[1:]  # [b1, b2]
prior_2d_cov = prior_cov[1:, 1:]  # 2x2 covariance for b1, b2

prior_dist_2d = multivariate_normal(mean=prior_2d_mean, cov=prior_2d_cov)
pos = np.dstack((B1, B2))
prior_density = prior_dist_2d.pdf(pos)

# Plot prior
fig = go.Figure(
    data=go.Heatmap(
        z=prior_density,
        x=b1_range,
        y=b2_range,
        colorscale="RdYlBu_r",
        colorbar=dict(title="Density"),
    )
)

fig.update_layout(
    title="Prior: p(b)",
    xaxis_title="b1 (MedInc coefficient)",
    yaxis_title="b2 (HouseAge coefficient)",
    width=700,
    height=600,
    yaxis=dict(scaleanchor="x", scaleratio=1),
)

fig.show()

## ✍️ Oefening 1: Likelihood Functie Implementeren

Implementeer de **likelihood** functie voor een enkele observatie $(x_i, y_i)$.

De likelihood voor een enkele observatie is:

$$
p(y_i | \pmb{x}_i, \pmb{b}, \sigma) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{1}{2\sigma^2}(y_i - \pmb{x}_i^T\pmb{b})^2\right)
$$

Voor numerieke stabiliteit berekenen we de **log-likelihood** en nemen daar de exponentiaal van:

$$
\log p(y_i | \pmb{x}_i, \pmb{b}, \sigma) = -\frac{1}{2\sigma^2}(y_i - \pmb{x}_i^T\pmb{b})^2 - \frac{1}{2}\log(2\pi\sigma^2)
$$

### Hints:
- De predictie is: $\hat{y}_i = b_0 + b_1 x_{i1} + b_2 x_{i2}$
- De residual is: $r_i = y_i - \hat{y}_i$
- De likelihood is proportioneel aan: $\exp\left(-\frac{\beta}{2} r_i^2\right)$, waarbij $\beta = 1/\sigma^2$

## ✍️ Oefening 2: Posterior na 1 Observatie

Bereken de **posterior** distributie na het observeren van het **eerste datapunt**.

Volgens de regel van Bayes:

$$
p(\pmb{b}|y_1) \propto p(y_1|\pmb{b}) \cdot p(\pmb{b})
$$

### Stappen:
1. Bereken de likelihood voor de eerste observatie
2. Vermenigvuldig met de prior
3. Normaliseer zodat de posterior integreert tot 1

### Visualisatie: Prior → Likelihood → Posterior (1 observatie)

## ✍️ Oefening 3: Sequentiële Update (5 observaties)

Nu gaan we **sequentieel** meer data toevoegen. Telkens gebruiken we de **vorige posterior als nieuwe prior**.

Dit illustreert het **incrementele leren** aspect van Bayesiaanse methoden.

### Algoritme:
```
posterior = prior
for i in range(n_observations):
    likelihood = compute_likelihood(x[i], y[i], ...)
    posterior = posterior * likelihood
    posterior = normalize(posterior)
```

### Visualisatie na 5 observaties

## ✍️ Oefening 4: Finale Posterior (alle 50 observaties)

Verwerk nu **alle 50 observaties** sequentieel en vergelijk de finale posterior met de OLS oplossing.

### Verwachting:
- De **posterior** zou moeten convergeren naar een scherpe piek rond de ware parameterwaarden
- De **MAP** (Maximum A Posteriori) schatting zou dicht bij de **OLS** schatting moeten liggen

### Vergelijking met OLS

Bereken de OLS oplossing en vergelijk met de MAP schatting.

### Finale Visualisatie

## 🎯 Bonus Oefening: Convergentie Visualiseren

Maak een plot die toont hoe de **MAP schattingen** evolueren als functie van het aantal observaties.

### Doel:
- Toon op de x-as het aantal observaties (1, 2, 3, ..., 50)
- Toon op de y-as de MAP schattingen voor $b_1$ en $b_2$
- Voeg horizontale lijnen toe voor de OLS waarden